# Reading lines in txt file

In [284]:
with open('input.txt') as f:
    lines = f.readlines()

In [285]:
lines

['00100\n',
 '10000\n',
 '10011\n',
 '00011\n',
 '\n',
 '0000000\n',
 '1100110\n',
 '1000000\n',
 '0000011\n',
 '1100000\n',
 '\n',
 '1000110\n',
 '0010000\n',
 '1010000\n',
 '1000110\n',
 '0010000\n',
 '\n',
 '1110000\n',
 '0110000\n',
 '0010011\n',
 '1110000\n',
 '0000100\n',
 '0000000']

# Empty list to store matrices

In [286]:
matrices = []
matrix_iteration = []

# Iterate over lines and append to list

In [287]:
for line in lines:
    if line == '\n': # Check for blank line
        matrices.append(matrix_iteration) # Add matrix to list of matrices
        matrix_iteration = [] # Reset matrix iteration
    else:
        matrix_iteration.append(line.strip()) # Add line to matrix iteration

## Add last matrix if file does not end with a blank line
if matrix_iteration:
    matrices.append(matrix_iteration)

In [288]:
matrices

[['00100', '10000', '10011', '00011'],
 ['0000000', '1100110', '1000000', '0000011', '1100000'],
 ['1000110', '0010000', '1010000', '1000110', '0010000'],
 ['1110000', '0110000', '0010011', '1110000', '0000100', '0000000']]

### Split character in each line (to columns) and convert to int using map

In [289]:
matrices = [[list(map(int, line)) for line in matrix] for matrix in matrices]

In [290]:
matrices

[[[0, 0, 1, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [0, 0, 0, 1, 1]],
 [[0, 0, 0, 0, 0, 0, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 1],
  [1, 1, 0, 0, 0, 0, 0]],
 [[1, 0, 0, 0, 1, 1, 0],
  [0, 0, 1, 0, 0, 0, 0],
  [1, 0, 1, 0, 0, 0, 0],
  [1, 0, 0, 0, 1, 1, 0],
  [0, 0, 1, 0, 0, 0, 0]],
 [[1, 1, 1, 0, 0, 0, 0],
  [0, 1, 1, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 1, 1],
  [1, 1, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0],
  [0, 0, 0, 0, 0, 0, 0]]]

# 1. Calculate the total number of islands in the matrix.

### Searching Neighbours (Function)

In [291]:
def search_neighbours(matrix, row, column, searched):
    # check if cell is within matrix
    if row < 0 or row >= len(matrix) or column < 0 or column >= len(matrix[0]):
        return
    # check if cell is already searched or is 0
    if searched[row][column] == True or matrix[row][column] == 0:
        return
    # mark cell as searched
    searched[row][column] = True
    # search neighbours
    search_neighbours(matrix, row-1, column, searched)  # search up one row
    search_neighbours(matrix, row+1, column, searched)  # search down one row
    search_neighbours(matrix, row, column-1, searched)  # search left one column
    search_neighbours(matrix, row, column+1, searched)  # search right one column

### Counting Islands (Function)

In [292]:
def count_islands(matrix):
    # create a matrix for searched cells using list comprehension
    searched = [[False for column in range(len(matrix[0]))] for row in range(len(matrix))]
    # count of islands found
    count = 0
    # iterate through matrix
    for row in range(len(matrix)):
        for column in range(len(matrix[0])):
            # check if cell is already searched
            if searched[row][column] == False and matrix[row][column] == 1:
                # search neighbouring cells
                search_neighbours(matrix, row, column, searched)
                # increment count when search is complete
                count += 1
    return count

### Iterating through matrices and counting islands

In [293]:
for _, matrix in enumerate(matrices):
    print(f"Matrix {_+1} has {count_islands(matrix)} islands")

Matrix 1 has 3 islands
Matrix 2 has 4 islands
Matrix 3 has 6 islands
Matrix 4 has 3 islands


# 2. Find the number of distinct islands

### Modifying the search function to store the coordinates of the island in a list

In [294]:
def search_neighbours_2(matrix, row, column, searched, matrix_coord):
    # check if cell is within matrix
    if row < 0 or row >= len(matrix) or column < 0 or column >= len(matrix[0]):
        return
    # check if cell is already searched or is 0
    if searched[row][column] == True or matrix[row][column] == 0:
        return
    # mark cell as searched
    searched[row][column] = True
    # add cell to matrix_coord
    matrix_coord.append((row, column))
    # search neighbours
    search_neighbours_2(matrix, row-1, column, searched, matrix_coord)  # search up one row
    search_neighbours_2(matrix, row+1, column, searched, matrix_coord)  # search down one row
    search_neighbours_2(matrix, row, column-1, searched, matrix_coord)  # search left one column
    search_neighbours_2(matrix, row, column+1, searched, matrix_coord)  # search right one column

### Normalizing the coordinates of the island to get the distinct islands (not rotated or reflected)

In [295]:
def normalize_island_coordinates(matrix_coord):
    # find minimum x and y
    min_x, min_y = matrix_coord[0]
    
    for x, y in matrix_coord:
        min_x = min(min_x, x) # update min_x if x is smaller
        min_y = min(min_y, y) # update min_y if y is smaller
    
    normalized_coords = []
    for x, y in matrix_coord:
        normalized_coords.append((x - min_x, y - min_y)) # add normalized coordinates to list
    
    return normalized_coords

### Modifying the count function to store the distinct islands in a set

In [296]:
def count_distinct_islands(matrix):
    # create a matrix for searched cells
    searched = [[False for column in range(len(matrix[0]))] for row in range(len(matrix))]
    # initialize set of island coordinates (to remove duplicates)
    coordinates = set()
    # iterate through matrix
    for row in range(len(matrix)):
        for column in range(len(matrix[0])):
            # check if cell is already searched
            if searched[row][column] == False and matrix[row][column] == 1:
                # list for island coordinates
                matrix_coord = []
                # search neighbours
                search_neighbours_2(matrix, row, column, searched, matrix_coord)
                # normalize coordinates to get rid of rotation and reflection
                normalized_coords = normalize_island_coordinates(matrix_coord)
                # sort to get rid of different orderings
                normalized_coords.sort()
                # add normalized_coords to set of coordinates
                coordinates.add(tuple(normalized_coords))
    return len(coordinates) # return number of distinct islands found in matrix (length of set)

### Iterating through matrices and counting distinct islands

In [297]:
for _, matrix in enumerate(matrices):
    count_distinct = count_distinct_islands(matrix)
    print(f"Matrix {_+1} has {count_distinct} distinct islands")

Matrix 1 has 3 distinct islands
Matrix 2 has 2 distinct islands
Matrix 3 has 3 distinct islands
Matrix 4 has 3 distinct islands


# 3. Find the number of closed islands.

### Modifying the search function to check if the island is closed (not touching the edge)

In [298]:
def search_neighbours_3(matrix, row, column, searched, closed):
    # check if cell is within matrix
    if row < 0 or row >= len(matrix) or column < 0 or column >= len(matrix[0]):
        return
    # check if cell is already searched or is 0
    if searched[row][column] == True or matrix[row][column] == 0:
        return
    # mark cell as searched
    searched[row][column] = True
    # check if cell is on the edge (left edge OR right edge OR top edge OR bottom edge)
    if row == 0 or row == len(matrix)-1 or column == 0 or column == len(matrix[0])-1:
        closed[0] = False
    # search neighbours
    search_neighbours_3(matrix, row-1, column, searched, closed)  # search up one row
    search_neighbours_3(matrix, row+1, column, searched, closed)  # search down one row
    search_neighbours_3(matrix, row, column-1, searched, closed)  # search left one column
    search_neighbours_3(matrix, row, column+1, searched, closed)  # search right one column

### Modify the count function to count the number of closed islands (not touching the edge)

In [299]:
def count_closed_islands(matrix):
    # create a matrix for searched cells
    searched = [[False for column in range(len(matrix[0]))] for row in range(len(matrix))]
    # count of closed islands
    count = 0
    # iterate through matrix
    for row in range(len(matrix)):
        for column in range(len(matrix[0])):
            # check if cell is already searched
            if searched[row][column] == False and matrix[row][column] == 1:
                # closed flag set to True by default.
                closed = [True] # using list to pass by reference as mutable (boolean is immutable)
                # search neighbours
                search_neighbours_3(matrix, row, column, searched, closed)
                # increment count if closed is True
                if closed[0] == True:
                    count += 1
    return count

In [300]:
for _, matrix in enumerate(matrices):
    print(f"Matrix {_+1} has {count_closed_islands(matrix)} closed islands")

Matrix 1 has 0 closed islands
Matrix 2 has 1 closed islands
Matrix 3 has 2 closed islands
Matrix 4 has 1 closed islands


# Results

In [301]:
for _, matrix in enumerate(matrices):
    print(f"Matrix {_+1} has {count_islands(matrix)} islands, {count_distinct_islands(matrix)} distinct islands, and {count_closed_islands(matrix)} closed islands")

Matrix 1 has 3 islands, 3 distinct islands, and 0 closed islands
Matrix 2 has 4 islands, 2 distinct islands, and 1 closed islands
Matrix 3 has 6 islands, 3 distinct islands, and 2 closed islands
Matrix 4 has 3 islands, 3 distinct islands, and 1 closed islands
